# Computation of SHAP values for prediction of risk from clinical factors

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shap


from sklearn.ensemble import (
    RandomForestRegressor,
)

from tqdm.notebook import tqdm


median_risk_df = pd.read_csv("median_risk.csv")
clinical_data = pd.read_csv("clinical_factors.csv")
clinical_vars = clinical_data.columns

In [2]:
X_train = clinical_data
y_train = pd.concat([median_risk_df[["nan weeks: none"]]]).rename(
    columns={"nan weeks: none": "risk"}
)

In [3]:
def _get_rf_sv(X_train, y_train):
    rf = RandomForestRegressor(oob_score=True)
    rf.fit(X_train, y_train)

    rf_exp = shap.TreeExplainer(rf)
    rf_sv = np.array(rf_exp.shap_values(X_train))
    rf_ev = np.array(rf_exp.expected_value)
    
    return rf_sv

In [4]:
rf_sv = [_get_rf_sv(X_train, y_train.values.ravel()) for i in tqdm(range(100))]

  0%|          | 0/100 [00:00<?, ?it/s]

In [5]:
rf_sv = np.mean(np.stack(rf_sv, axis=-1), axis=-1)

# Shap values:

In [6]:
rf_sv

array([[-8.49024710e-03, -8.71296086e-04, -6.47474942e-02, ...,
        -1.42667189e-04, -1.51923514e-03,  1.45651195e-03],
       [-5.18194584e-03, -1.52977863e-02, -4.82676436e-02, ...,
        -3.99186750e-05, -1.12213137e-03,  1.78269093e-03],
       [ 3.93476917e-03,  2.99417275e-03, -5.03332666e-02, ...,
         2.02970748e-05, -7.10770820e-04,  2.20706040e-03],
       ...,
       [ 3.42707789e-02,  3.07731470e-02,  2.46984453e-02, ...,
         1.89593874e-05,  1.02992236e-04,  1.73669074e-03],
       [ 4.53867375e-02,  5.35913220e-02,  3.04151662e-01, ...,
         2.71657422e-05, -4.03998629e-04,  1.47563093e-03],
       [-1.09633110e-02, -2.73345769e-02, -4.16369258e-02, ...,
        -1.67793132e-05, -4.00506796e-04,  2.68262093e-03]])